In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #visualize data
import seaborn as sns # visualize data with more appealing 
import scipy.stats as stats # 
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression #machine learning
from sklearn.model_selection import train_test_split 

Loading California Data 

In [2]:
file_path = "housing.csv"

In [ ]:
data = pd.read_csv(file_path)
data

In [ ]:
data.columns

In [ ]:
data.head(10)

In [ ]:
data.info()

In [ ]:
data["ocean_proximity"].unique()

Missing Data Analysis

In [ ]:
# Check for missing values
missing_values = data.isnull().sum()

# calculate the missing data percentage in each column
missing_percentage = (missing_values / len(data)) * 100

# Display the missing data statistics
print("Missing values in each column:\n",missing_values)
print("Percentage of missing data:\n",missing_percentage)

In [ ]:
# Remove rows with missing values
data_cleaned = data.dropna()

print("Missing valuesin each column after cleaning:")
print(data_cleaned.isnull().sum())

In [ ]:
data.describe()

In [ ]:

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10,6))
sns.histplot(data_cleaned['median_house_value'],color='forestgreen',kde=True)
plt.title('Distribution of Median House Value')
plt.xlabel('Median house value')
plt.ylabel('Frequency')
plt.show()

Using InterQuantile Range to Remove Outliers

In [ ]:
Q1 = data_cleaned['median_house_value'].quantile(0.25)
Q3 = data_cleaned['median_house_value'].quantile(0.75)
IQR = Q3-Q1

#Define the bounds for the outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#Remove outliers
data_no_outliers_1 = data_cleaned[(data_cleaned['median_house_value'] >= lower_bound) & (data_cleaned['median_house_value'] <= upper_bound)]

#check the shpae of the data before and after removal of outliers
print("Original data shape",data_cleaned.shape)
print("New data shape without outliers",data_no_outliers_1.shape)

BoxPlot for outlier detection

Outliers in Median Income
        |
        |

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x=data_no_outliers_1['median_income'],color='beige')
plt.title('Outlier Analysis in Median Income')
plt.xlabel('Median Income')
plt.show()

In [ ]:
Q1 = data_no_outliers_1['median_income'].quantile(0.25)
Q3 = data_no_outliers_1['median_income'].quantile(0.75)
IQR = Q3-Q1

#Define the bounds for the outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#Remove outliers
data_no_outliers_2 = data_no_outliers_1[(data_no_outliers_1['median_income']>= lower_bound) & (data_no_outliers_1['median_income'] <= upper_bound)]

#check the shpae of the data before and after removal of outliers
print("Original data shape",data_no_outliers_1.shape)
print("New data shape without outliers",data_no_outliers_2.shape)

In [ ]:
data = data_no_outliers_2
data

Correlation Heatmap

In [ ]:
plt.figure(figsize=(12,8))
#data.select_dtypes(include=['number']) specifies to select only numerical to avoid conflict with string datatypes while plotting heatmaps
sns.heatmap(data.select_dtypes(include=['number']).corr(),annot=True,cmap='Greens')
plt.title('correlation Heatmap of Housing Data')
plt.show()


the total bedrooms column is being dropped below becuase it is causing high dependency with two independent variable which will cause a deviation while training the models

In [ ]:
data = data.drop("total_bedrooms",axis=1)
data.columns

In [ ]:
# Unique value count for categorical data
for column in ['ocean_proximity']:
    print(f"Unique values in {column}:",data[column].unique())

String Data Categorization to Dummy variables

In [19]:
ocean_proximity_dummies = pd.get_dummies(data['ocean_proximity'], prefix='ocean_proximity').astype(int)
data = pd.concat([data.drop("ocean_proximity", axis=1), ocean_proximity_dummies],axis =1)


In [ ]:
print(ocean_proximity_dummies.dtypes)

In [ ]:
ocean_proximity_dummies

Split and Train Dataset

In [ ]:
data = data.drop("ocean_proximity_ISLAND",axis=1)
data.columns

In [ ]:
data.head(1)
data.columns

In [ ]:
# Define features (Independent Variables) and target (dependent variables)
features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population', 'households', 'median_income', 'median_house_value',
       'ocean_proximity_<1H OCEAN', 'ocean_proximity_INLAND',
       'ocean_proximity_NEAR BAY', 'ocean_proximity_NEAR OCEAN']
target = ["median_house_value"]

X = data[features]
y = data[target]

# Split the data in to test and trianing
# Test size specifies what portion of data should use for training
# random_state ensures reproducibility of your split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1111)

# Check the size of the splits
print(f'Training set size: {X_train.shape[0]} samples')
print(f'Test set size: {X_test.shape[0]} samples')

Training

In [25]:
# Adding a constant to the predictors because statsmodels OLS doesn't include it by default
X_train_constant = sm.add_constant(X_train)

In [ ]:
# Fit the OLS model
model_fitted = sm.OLS(y_train,X_train_constant).fit()

# Printing Summary
print(model_fitted.summary())

Prediction/Testing  

In [27]:
# Adding a constant to the test predictors
X_test_const = sm.add_constant(X_test)

# Making predictions on the test set
test_predictions = model_fitted.predict(X_test_const)

In [ ]:
test_predictions

Checking OLS Assumptions

Assumption 1: Linearity

In [ ]:
# Scatter plot for observed vs predicted values on test data
plt.scatter(y_test, test_predictions, color='forestgreen')
plt.xlabel('Observed Values')
plt.ylabel('Predicted Values')
plt.title('Observed vs Predicted Values on Test Data')
plt.plot(y_test, y_test, color='black') # line for perfect prediction (true values)
plt.show()

Assumption 2: Random Sample

In [ ]:
# Calculate the mean of the residuals
mean_residuals = np.mean(model_fitted.resid)

print(f"The mean of the residuals is {np.round(mean_residuals,2)}")

In [ ]:
# Plotting residuals
plt.scatter(model_fitted.fittedvalues, model_fitted.resid, color='forestgreen')
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

Assumption 3: Exogeneity

In [ ]:
# Calculate the residuals
residuals = model_fitted.resid

# Check for correlation between residuals and each predictor
for column in X_train.columns:
    corr_coefficient = np.corrcoef(X_train[column], residuals)[0,1]
    print(f'Correlation between residuals and {column}: {np.round(corr_coefficient,2)}')


Asumption 4: Homoskedasticity

In [ ]:
# Plotting residuals
plt.scatter(model_fitted.fittedvalues, model_fitted.resid, color='forestgreen')
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

Train/Test/Evaluation with Sklearn


Scaling the Data

In [34]:
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Apply the same transformation to the test data
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

# Create and fit the model

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
print()

# Make the predictions on the scaled test data
y_pred = lr.predict(X_test_scaled)

#Calculate MSE and RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)

# Output the performance metrics
print(f'MSE on Test Set: {mse}')
print(f'RMSE on Test Set: {rmse}')

In [ ]:
print(y_test,y_pred)


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)

print(r2)
